# K-Means

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
from operator import itemgetter

In [61]:
from typing import Union, Literal

class KMeans:
    """
    Perform KMeans clustering on a dataset.
    """

    def __init__(self,
                 algorithm : Literal['lloyd', 'extended-hartigan', 'safe-hartigan', 'hartigan'] = 'lloyd',
                 init : Literal['random', 'random-data', 'kmeans++', 'greedy'] = 'random',
                 seed : Union[int, None] = None):
        """
        Initialize the KMeans object.

        Parameters
        ----------
        algorithm : {'lloyd', 'extended-hartigan', 'safe-hartigan', 'hartigan', 'extended-hartigan-2', 'safe-hartigan-2', 'hartigan-2'}
            Algorithm to use. Either 'lloyd' or 'extended-hartigan' or 'safe-hartigan' or 'hartigan' or 'extended-hartigan-2'

        init : {'random', 'random-data', 'kmeans++', 'greedy'}
            Initialization method. Either 'random' or 'random-data' or 'kmeans++' or 'greedy'

        seed : int
            Seed for random generator
        """

        assert algorithm in ['lloyd', 'extended-hartigan', 'safe-hartigan', 'hartigan'], "algorithm must be either 'lloyd' or 'hartigan'"
        assert init in ['random', 'random-data', 'kmeans++', 'greedy'], "init must be either 'random', 'random-data', 'kmeans++' or 'greedy'"
        assert seed is None or isinstance(seed, int), "seed must be an int or None"

        self.algorithm = algorithm
        self.init = init
        self.seed = seed

        self.data = None
        self.k = None
        self.centroids = None
        self.y_pred = None


    def fit(self, data : np.ndarray, k : int, debug=False):
        """
        Fit the model to the data.

        Parameters
        ----------
        data : np.ndarray
            nxd DataFrame of n samples with d features
        k : int
            Number of clusters

        Returns
        -------
        np.ndarray
            Array of shape (k, d) with cluster centroids
        np.ndarray
            Array of length n with cluster assignments for each sample
        """

        assert isinstance(data, np.ndarray), "data must be a numpy array"
        assert len(data.shape) == 2, "data must be a 2D array"
        assert isinstance(k, int), "k must be an int"
        assert k <= len(data), "k must be at most the number of samples"

        self.data = data
        self.k = k

        np.random.seed(self.seed)

        # initialize centroids
        self._init_centroids(debug)
        debug and print('initial centroids:\n', self.centroids)

        ## TODO: implement clustering algorithm

        if self.algorithm == 'lloyd':
            self._lloyd(debug)
        elif self.algorithm == 'extended-hartigan':
            return NotImplemented
            self._extended_hartigan(debug)
        elif self.algorithm == 'safe-hartigan':
            return NotImplemented
            self._safe_hartigan(debug)
        
        print('final centroids:\n', self.centroids)
        print('final y_pred:', self.y_pred)


    ## TODO: implement predict method
    def predict(self):
        return NotImplemented


    def _init_centroids(self, debug=False):
        """
        Initialize the centroids.
        """

        if self.init == 'random':

            # choose k random data points as initial centroids
            idx = np.random.choice(self.data.shape[0], self.k, replace=False)
            self.centroids = self.data[idx]

        elif self.init == 'random-data':

            # assign each data point to a random cluster
            clusters = np.random.choice(self.k, self.data.shape[0])

            # check that at least one point is assigned to each cluster
            while len(set(clusters)) < self.k:
                clusters = np.random.choice(self.k, self.data.shape[0])
            self.y_pred = clusters
            self.centroids = self._move_centroids(debug)

        elif self.init == 'kmeans++':
    
            # choose first centroid randomly
            centroids = np.zeros((self.k, self.data.shape[1]))
            centroids[0] = self.data[np.random.choice(self.data.shape[0], 1, replace=False)[0]]
            debug and print('centroids:\n', centroids)

            # iterate over remaining k-1 centroids
            for i in range(1, self.k):
                debug and print('iteration', i)

                # calculate distance squared of each point to closest centroid
                dist = np.array([min([np.linalg.norm(c-x)**2 for c in centroids[:i]]) for x in self.data])

                # probabilities are given by the normalized distance squared
                probs = dist / dist.sum()
                debug and print('probs:', probs)

                # cumulate probabilities
                cumprobs = probs.cumsum()

                # choose next centroid randomly based on cumulated probabilities
                r = np.random.rand()
                debug and print('r:', r)
                for j, p in enumerate(cumprobs):
                    if r < p:
                        break

                centroids[i] = self.data[j]
                debug and print('centroids:\n', centroids)

            self.centroids = centroids

        elif self.init == 'greedy':
            return NotImplemented


    def _lloyd(self, debug=False):
        """
        Lloyd's algorithm for k-means clustering.
        """

        debug and print('Running Lloyd\'s algorithm...')
        while True:

            # assign each data point to the closest centroid
            self.y_pred = self._assign_clusters(debug)
            debug and print('y_pred:', self.y_pred)

            # move centroids to the mean of their cluster
            new_centroids = self._move_centroids(debug)

            # check for convergence
            if np.allclose(self.centroids, new_centroids):
                break

            self.centroids = new_centroids


    def _move_centroids(self, debug=False):
        """
        Move the centroids to the mean of their cluster.
        """

        debug and print('y_pred:', self.y_pred)
        debug and print('data:\n', self.data)
        debug and print('centroids_before:', self.centroids)

        centroids = np.zeros((self.k, self.data.shape[1]))
        for centroid_id in range(self.k):
            cluster_points = self.data[self.y_pred == centroid_id]
            
            # if centroid has no points assigned to it, reassign it randomly
            if len(cluster_points) == 0:
                debug and print(f"Centroid {centroid_id} is empty. Reassigning.")
                centroids[centroid_id] = self.data[np.random.choice(len(self.data))]
            else:
                centroids[centroid_id] = np.mean(cluster_points, axis=0)

        debug and print('centroids_after:\n', centroids)

        return centroids


    def _assign_clusters(self, debug=False):
        """
        Assign each data point to the closest centroid.
        """

        y_pred = np.zeros(len(self.data), dtype=int)
        for i, x in enumerate(self.data):
            debug and print([np.linalg.norm(x-c)**2 for c in self.centroids])
            y_pred[i] = np.argmin([np.linalg.norm(x-c)**2 for c in self.centroids])

        return y_pred


In [27]:
def accuracy(y_true : np.ndarray, y_pred : np.ndarray):
    """
    Compute the accuracy of the clustering.
    
    Parameters
    ----------
    y_true : np.ndarray
        True labels of the samples
    y_pred : np.ndarray
        Predicted labels of the samples
    
    Returns
    -------
    float
        Accuracy of the clustering through Hungarian algorithm
    """
    
    assert isinstance(y_true, np.ndarray), "y_true must be a numpy array"
    assert isinstance(y_pred, np.ndarray), "y_pred must be a numpy array"

    # create C matrix
    n_classes = max(max(y_true), max(y_pred)) + 1
    C = np.zeros((n_classes, n_classes), dtype=int)
    for true_label, pred_label in zip(y_true, y_pred):
        C[true_label, pred_label] += 1
    
    # Solve assignment problem
    row_ind, col_ind = linear_sum_assignment(-C)
    
    # Calculate accuracy
    matched = C[row_ind, col_ind].sum(axis=0)
    accuracy = matched / len(y_true)
    return accuracy

In [205]:
a = np.array([[1, 2], [3, 4], [11, 12], [13, 14], [19, 20]])

kmeans = KMeans(algorithm='lloyd', init='kmeans++')
kmeans.fit(a, 3, True)
print(type(kmeans.y_pred))

accuracy(np.array([0, 0, 1, 1, 2]), kmeans.y_pred)

centroids:
 [[1. 2.]
 [0. 0.]
 [0. 0.]]
iteration 1
probs: [0.         0.00699301 0.17482517 0.25174825 0.56643357]
r: 0.8589837727406298
centroids:
 [[ 1.  2.]
 [19. 20.]
 [ 0.  0.]]
iteration 2
probs: [0.         0.03846154 0.61538462 0.34615385 0.        ]
r: 0.31815005408046226
centroids:
 [[ 1.  2.]
 [19. 20.]
 [11. 12.]]
initial centroids:
 [[ 1.  2.]
 [19. 20.]
 [11. 12.]]
Running Lloyd's algorithm...
[np.float64(0.0), np.float64(648.0), np.float64(200.00000000000003)]
[np.float64(8.000000000000002), np.float64(512.0000000000001), np.float64(128.00000000000003)]
[np.float64(200.00000000000003), np.float64(128.00000000000003), np.float64(0.0)]
[np.float64(287.99999999999994), np.float64(71.99999999999999), np.float64(8.000000000000002)]
[np.float64(648.0), np.float64(0.0), np.float64(128.00000000000003)]
y_pred: [0 0 2 2 1]
y_pred: [0 0 2 2 1]
data:
 [[ 1  2]
 [ 3  4]
 [11 12]
 [13 14]
 [19 20]]
centroids_before: [[ 1.  2.]
 [19. 20.]
 [11. 12.]]
centroids_after:
 [[ 2.  3.]
 [19

np.float64(1.0)

In [329]:
a = np.array([[0,0], [1,1], [2,2], [3,3], [4,4], [5,5], [6,6], [7,7], [8,8], [9,9]])
for i in range(1,10):
    print(i, a[:i])

1 [[0 0]]
2 [[0 0]
 [1 1]]
3 [[0 0]
 [1 1]
 [2 2]]
4 [[0 0]
 [1 1]
 [2 2]
 [3 3]]
5 [[0 0]
 [1 1]
 [2 2]
 [3 3]
 [4 4]]
6 [[0 0]
 [1 1]
 [2 2]
 [3 3]
 [4 4]
 [5 5]]
7 [[0 0]
 [1 1]
 [2 2]
 [3 3]
 [4 4]
 [5 5]
 [6 6]]
8 [[0 0]
 [1 1]
 [2 2]
 [3 3]
 [4 4]
 [5 5]
 [6 6]
 [7 7]]
9 [[0 0]
 [1 1]
 [2 2]
 [3 3]
 [4 4]
 [5 5]
 [6 6]
 [7 7]
 [8 8]]


In [323]:
a = np.array([[0.,0.], [1.,1.], [2.,2.]])

# only positive entries
print(np.all(a != 0, axis=1))
a[np.all(a != 0, axis=1)]


[False  True  True]


array([[1., 1.],
       [2., 2.]])

In [169]:
np.random.choice(5, 10)

array([2, 3, 4, 3, 4, 3, 1, 4, 3, 1], dtype=int32)

In [3]:
from sklearn.cluster import KMeans as KMeans_sklearn

KMeans = KMeans_sklearn()
KMeans.fit = KMeans_sklearn.fit()